<a href="https://colab.research.google.com/github/MB-13/Brain-tumor-classification-using-Fine-Tuning/blob/main/Brain_Tumor_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri
!kaggle datasets download -d denizkavi1/brain-tumor

 65% 56.0M/86.8M [00:00<00:00, 209MB/s] 
100% 86.8M/86.8M [00:00<00:00, 214MB/s]
100% 697M/700M [00:11<00:00, 60.8MB/s]
100% 700M/700M [00:11<00:00, 62.2MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/brain-tumor-classification-mri.zip')
zip_ref_2 = zipfile.ZipFile('/content/brain-tumor.zip')
zip_ref.extractall('/content')
zip_ref_2.extractall('/content')
zip_ref.close()
zip_ref_2.close()

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tp
from tensorflow import keras
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_val_score
from keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,Dropout,Flatten

**Gathering** **Dataset**

In [ ]:
import PIL

In [ ]:
data_dir = '/content/Training'
test_data_dir = '/content/Testing'
data_meni_1 = '/content/1'
data_glio_2 = '/content/2'
data_pitu_3 = '/content/3'

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
test_data_dir = pathlib.Path(test_data_dir)
data_meni_1 = pathlib.Path(data_meni_1)
data_glio_2 = pathlib.Path(data_glio_2)
data_pitu_3 = pathlib.Path(data_pitu_3)
print(data_dir)

/content/Training


In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count
image_count_t = len(list(test_data_dir.glob('*/*.jpg')))
image_count_t
len(list(data_meni_1.glob('*.png')))

708

In [ ]:
# print(len(list(data_dir.glob('glioma_tumor/*'))))
# print(len(list(data_dir.glob('meningioma_tumor/*'))))
# print(len(list(data_dir.glob('no_tumor/*'))))
# print(len(list(data_dir.glob('pituitary_tumor/*'))))

In [ ]:
# print(len(list(test_data_dir.glob('glioma_tumor/*'))))
# print(len(list(test_data_dir.glob('meningioma_tumor/*'))))
# print(len(list(test_data_dir.glob('no_tumor/*'))))
# print(len(list(test_data_dir.glob('pituitary_tumor/*'))))

In [ ]:
brain_images_dict = {
    'glioma_tumor' : list(data_dir.glob('glioma_tumor/*')),
    'meningioma_tumor' : list(data_dir.glob('meningioma_tumor/*')),
    'no_tumor' : list(data_dir.glob('no_tumor/*')),
    'pituitary_tumor' : list(data_dir.glob('pituitary_tumor/*'))
}

sec_data_images = {
    'meningioma_tumor' : list(data_meni_1.glob('*.png')),
    'glioma_tumor' : list(data_glio_2.glob('*.png')),
    'pituitary_tumor' : list(data_pitu_3.glob('*.png'))
}

test_brain_images_dict = {
    'glioma_tumor' : list(test_data_dir.glob('glioma_tumor/*')),
    'meningioma_tumor' : list(test_data_dir.glob('meningioma_tumor/*')),
    'no_tumor' : list(test_data_dir.glob('no_tumor/*')),
    'pituitary_tumor' : list(test_data_dir.glob('pituitary_tumor/*'))
}

brain_label_dict = {
    'glioma_tumor' : 0,
    'meningioma_tumor' : 1,
    'no_tumor' : 2,
    'pituitary_tumor' : 3
}

sec_data_label = {
    'meningioma_tumor' : 1,
    'glioma_tumor' : 0,
    'pituitary_tumor' : 3
}

In [ ]:
import cv2
str(brain_images_dict['no_tumor'][5])
str(sec_data_images['pituitary_tumor'][1])

'/content/3/1430.png'

In [ ]:
df = pd.DataFrame(columns=["image_paths", "label"])

In [ ]:
X,y = [],[]

for tumor_class,images in brain_images_dict.items():
  for image in images:
    img = str(image)
    X.append(img)
    y.append(brain_label_dict[tumor_class])

for tumor_class,images in test_brain_images_dict.items():
  for image in images:
    img = str(image)
    X.append(img)
    y.append(brain_label_dict[tumor_class])

for tumor_class,images in sec_data_images.items():
  for image in images:
    img = str(image)
    X.append(img)
    y.append(sec_data_label[tumor_class])

In [ ]:
for i in range(len(X)):
  df.loc[i] = [X[i], str(y[i])]

In [ ]:
df.shape

(6328, 2)

In [ ]:
df.sample(10)

,image_paths,label
2781,/content/Training/pituitary_tumor/p (82).jpg,3
5633,/content/3/2044.png,3
6148,/content/3/1729.png,3
2145,/content/Training/pituitary_tumor/p (215).jpg,3
631,/content/Training/glioma_tumor/gg (21).jpg,0
4839,/content/2/106.png,0
858,/content/Training/meningioma_tumor/m1(8).jpg,1
5933,/content/3/1013.png,3
1694,/content/Training/no_tumor/image(30).jpg,2
515,/content/Training/glioma_tumor/gg (505).jpg,0


In [ ]:
# convert_dict = {'image_paths': object,
#                 'label': categorical
#                 }
# df = df.astype(convert_dict)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6328 entries, 0 to 6327
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image_paths  6328 non-null   object
 1   label        6328 non-null   object
dtypes: object(2)
memory usage: 148.3+ KB


In [ ]:
# making training and testing data

In [ ]:
train, test = train_test_split(df, test_size= 0.2, random_state= 42)
training, validation = train_test_split(train, test_size= 0.25, random_state= 42, shuffle = True)

In [ ]:
target_size = (224,224)

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.4,1.2],
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(
    rescale=1.0/255
)

In [ ]:
def create_dataset(df,is_training=False):
  if is_training:
    datagen = train_datagen
  else:
    datagen = valid_test_datagen

  return datagen.flow_from_dataframe(
      df,
      x_col='image_paths',
      y_col='label',
      class_mode='categorical',
      shuffel=is_training,
      target_size=target_size)

In [ ]:
train_dataset = create_dataset(training,is_training=True)
test_dataset = create_dataset(test)
validation_dataset = create_dataset(validation)

Found 3796 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.
Found 1266 validated image filenames belonging to 4 classes.


In [ ]:
base = keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

base.trainable = False
for layer in base.layers[-7:]:
  layer.trainable = True
for layer in base.layers:
  print(layer.trainable)

flatten = Flatten()(base.output)
x = Dense(473,activation='relu')(flatten)
x = Dropout(0.08179070851723078)(x)
output = Dense(4,activation='softmax')(x)

model = keras.Model(base.input,output)
model.summary()

False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True


In [ ]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0008356345045084528),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
history = model.fit(train_dataset,epochs=100,validation_data=validation_dataset,callbacks=[early_stop])

Epoch 1/100
119/119 [==============================] - 118s 876ms/step - loss: 1.2398 - accuracy: 0.4038 - val_loss: 1.0581 - val_accuracy: 0.5047
Epoch 2/100
119/119 [==============================] - 94s 790ms/step - loss: 1.0697 - accuracy: 0.5211 - val_loss: 0.8599 - val_accuracy: 0.6564
Epoch 3/100
119/119 [==============================] - 95s 799ms/step - loss: 0.9596 - accuracy: 0.5830 - val_loss: 0.6996 - val_accuracy: 0.7283
Epoch 4/100
119/119 [==============================] - 96s 803ms/step - loss: 0.8792 - accuracy: 0.6283 - val_loss: 0.6828 - val_accuracy: 0.7275
Epoch 5/100
119/119 [==============================] - 91s 762ms/step - loss: 0.8133 - accuracy: 0.6628 - val_loss: 0.5696 - val_accuracy: 0.7765
Epoch 6/100
119/119 [==============================] - 89s 751ms/step - loss: 0.7452 - accuracy: 0.6947 - val_loss: 0.4978 - val_accuracy: 0.8183
Epoch 7/100
119/119 [==============================] - 89s 750ms/step - loss: 0.6655 - accuracy: 0.7366 - val_loss: 0.5075 

**Parameter** **tunung** **using** **Optuna**

In [ ]:
# !pip install optuna
# import optuna

In [ ]:
# def create_model(trial):
#   loaded_model = keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

#   N = trial.suggest_int("unfreeze_layers",1,17)
#   for layer in loaded_model.layers[-N:]:
#     layer.trainable = True

#   learning_rate = trial.suggest_float('learning_rate',1e-5,1e-2,log=True)

#   model = Sequential()
#   for layer in loaded_model.layers[:-1]:
#     model.add(layer)
#     model.add(Dropout(0.25))
#     model.add(Dense(4,activation='softmax'))

#     optimizer = keras.optimizers.SGD(learning_rate=learning_rate,momentum=0.9)
#     model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#     return model


In [ ]:
# from keras.models import Model
# def objective(trial):
#   base_model = keras.applications.VGG16(weights='imagenet',include_top = False,input_shape=(224,224,3))

#   for layer in base_model.layers[:-trial.suggest_int('unfreeze_layers',1,10)]:
#     layer.trainable = False
#   base_model_output = base_model.output
#   x = Flatten()(base_model_output)
#   for _ in range(trial.suggest_int('fc_layers',1,5)):
#     x = Dense(trial.suggest_int('fc_units',64,512),activation='relu')(x)
#   output = Dense(4,activation='softmax')(x)

#   model = Model(inputs=base_model.input,outputs=output)

#   model.compile(optimizer=keras.optimizers.SGD(learning_rate = trial.suggest_float('learning_rate',1e-5,1e-2,log=True),momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
#   model.fit(train_dataset,epochs=5,batch_size=32,validation_data=validation_dataset)
#   val_loss,val_accuracy = model.evaluate(validation_dataset)
#   return val_accuracy
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20)

In [ ]:
# best_params = study.best_params
# print("Best Hyperparameters:", best_params)


In [ ]:
model.save("bt_fun.h5")
from google.colab import files
files.download("bt_fun.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>